In [9]:
import json, os
from openai import OpenAI

data_path = "../data/QnA_data/"
# filename = "QnA_test.json"
# filename = "QnA_summary_hin_India_election_2024_100.json"
# filename = "auto_QnA_summary_zho_Territorial_disputes_in_the_South_China_Sea_100.json"
# filename = "v2_auto_QnA_summary_eng_Gaza_100.json"
filename = "v2_auto_QnA_summary_zho_Gaza_100.json"
source_filename = data_path + filename
des_filename = source_filename


# Load articles
with open(source_filename, 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Load questions
with open(data_path+'formated_output_questions_v2.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)



In [10]:
def get_answers(article, questions, model):
    article_title = article.get('title')
    article_content = article.get('body')
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():
    pre = "You are an AI trained to understand articles and generate concise answers to multiple-choice questions based on the content. \
        Please read the following article carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \n\n"

    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [11]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ';':
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(';')}



In [12]:
index = 61
article = articles[index]
print(len(article['answers']))
print(article['answers'])


51
{'2': 'B', '3': 'B', '4': 'B', '5': 'E', '6': 'E', '7': 'E', '8': 'C', '9': 'E', '10': 'E', '11': 'E', '12': 'E', '13': 'E', '14': 'E', '15': 'B', '16': 'E', '17': 'E', '18': 'E', '19': 'E', '20': 'E', '21': 'E', '22': 'E', '23': 'E', '24': 'A', '25': 'D', '26': 'A', '27': 'C', '28': 'A', '29': 'D', '30': 'E', '31': 'E', '32': 'E', '33': 'C', '34': 'A', '35': 'E', '36': 'E', '37': 'E', '38': 'E', '39': 'E', '40': 'E', '41': 'B', '42': 'E', '43': 'E', '44': 'B', '45': 'E', '46': 'E', '47': 'E', '48': 'E', '49': 'E', '50': 'B', '51': 'E', '52': 'E'}


In [13]:
if 'answers' in article:
    del article['answers']

print(article)

{'uri': '8169217873', 'lang': 'zho', 'isDuplicate': True, 'date': '2024-06-09', 'time': '13:32:41', 'dateTime': '2024-06-09T13:32:41Z', 'dateTimePub': '2024-06-09T13:30:33Z', 'dataType': 'news', 'sim': 0, 'url': 'https://udn.com/news/story/123777/8020329', 'title': '以色列為救人質突襲 今持續猛攻加薩中部...哈瑪斯：迄今釀274死 | 聯合新聞網', 'body': '加薩居民及哈瑪斯表示，以色列軍隊昨天為營救人質進行突襲，迄今造成274名巴勒斯坦人死亡，以軍今天繼續猛攻加薩中部，戰車進一步深入南部城市拉法，試圖封鎖部分地區。\n\n路透社報導，巴勒斯坦武裝組織哈瑪斯（Hamas）治下的加薩（Gaza）衛生部今天更新傷亡人數時指出，巴勒斯坦人遇害者已從昨天通報的210人，攀升至274人，另有698人受傷。\n\n以色列特種突擊隊昨天闖進加薩中部人口稠密的紐瑟拉特（Nuseirat）難民營，營救去年10月起遭哈瑪斯武裝分子挾持的4名人質。\n\n以色列軍方發表聲明指出，以軍繼續在加薩中部布賴吉（El-Bureij）以東及迪爾巴拉市（Deir al-Balah）行動，已擊斃數名巴勒斯坦槍手，並且摧毀武裝分子基礎設施。\n\n根據受困家中的居民，以色列戰車今天開進兩個新地區，顯然為了完成對整個拉法（Rafah）東側的包圍，與哈瑪斯領導的武裝團體爆發衝突。', 'source': {'uri': 'udn.com', 'dataType': 'news', 'title': 'UDN', 'location': {'type': 'place', 'label': {'eng': 'Taipei'}, 'country': {'type': 'country', 'label': {'eng': 'Republic of China (Taiwan)'}}}, 'locationValidated': False}, 'authors': [], 'image': 'https://pgw.udn.com.tw

In [14]:
answers = get_answers(article, questions, model="gpt-4o")
pure_answers = answers.choices[0].message.content
print(pure_answers)

# convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
processed_answers = convert_string_to_dict(pure_answers)
article['answers'] = processed_answers
print(len(article['answers']))


Proceeding article 以色列為救人質突襲 今持續猛攻加薩中部...哈瑪斯：迄今釀274死 | 聯合新聞網
1B;2D;3B;4B;5E;6E;7E;8C;9E;10E;11E;12E;13E;14E;15B;16E;17E;18E;19E;20E;21E;22E;23E;24A;25D;26A;27C;28A;29D;30E;31E;32E;33C;34A;35E;36E;37E;38E;39E;40E;41B;42E;43E;44B;45E;46E;47E;48D;49E;50B;51E;52E
52


In [15]:
print(articles[index]['answers'])

{'1': 'B', '2': 'D', '3': 'B', '4': 'B', '5': 'E', '6': 'E', '7': 'E', '8': 'C', '9': 'E', '10': 'E', '11': 'E', '12': 'E', '13': 'E', '14': 'E', '15': 'B', '16': 'E', '17': 'E', '18': 'E', '19': 'E', '20': 'E', '21': 'E', '22': 'E', '23': 'E', '24': 'A', '25': 'D', '26': 'A', '27': 'C', '28': 'A', '29': 'D', '30': 'E', '31': 'E', '32': 'E', '33': 'C', '34': 'A', '35': 'E', '36': 'E', '37': 'E', '38': 'E', '39': 'E', '40': 'E', '41': 'B', '42': 'E', '43': 'E', '44': 'B', '45': 'E', '46': 'E', '47': 'E', '48': 'D', '49': 'E', '50': 'B', '51': 'E', '52': 'E'}


In [16]:
# Save results to a JSON file
with open(des_filename, 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)